In [18]:
import json
import numpy
import scipy
import utils.molecule_tools as mol
import utils.qonversion_tools as qonvert
import utils.linalg_tools as la
import utils.bit_tools as bit
import fermions.yaferp.misc.tapering as taper
import fermions.yaferp.general.fermions as fermions
import fermions.yaferp.general.sparseFermions as sparseFermions

In [30]:
with open('data/molecule_data.json', 'r') as json_file:
    molecule_data = json.load(json_file)
print(len(molecule_data))
list(molecule_data.keys())

32


['H2_3-21G_SINGLET',
 'H6_STO-3G_SINGLET',
 'H2_6-31G_SINGLET',
 'H2_6-311G_SINGLET',
 'H3+_STO-3G_SINGLET',
 'H3+_3-21G_SINGLET',
 'H3+_STO-3G_SINGLET_alt',
 'HeH+_3-21G_SINGLET',
 'HeH+_3-21G_SINGLET_alt',
 'HeH+_6-311G_SINGLET',
 'H2O_STO-3G_SINGLET',
 'H2O_STO-3G_SINGLET_alt',
 'BeH+_STO-3G_SINGLET',
 'LiH_STO-3G_SINGLET',
 'CH+_STO-3G_SINGLET',
 'HF_STO-3G_SINGLET',
 'B+_STO-3G_SINGLET',
 'N+_STO-3G_SINGLET',
 'OH+_STO-3G_SINGLET',
 'O_STO-3G_SINGLET',
 'CH2_STO-3G_SINGLET',
 'BeH2_STO-3G_SINGLET',
 'Be_STO-3G_SINGLET',
 'C_STO-3G_SINGLET',
 'NH_STO-3G_SINGLET',
 'Ne_STO-3G_SINGLET',
 'F+_STO-3G_SINGLET',
 'Li+_STO-3G_SINGLET',
 'BH_STO-3G_SINGLET',
 'NeH+_STO-3G_SINGLET',
 'NH2+_STO-3G_SINGLET',
 'BH2+_STO-3G_SINGLET']

In [20]:
molecules = ['LiH_STO-3G_SINGLET','HF_STO-3G_SINGLET', 'C_STO-3G_SINGLET', 'Be_STO-3G_SINGLET']

In [21]:
def hfDegenerateGroundState(oplist):
    oplistDiag = [x for x in oplist if 2 not in x[1] and 1 not in x[1]]
    mat = sparseFermions.commutingOplistToMatrix(oplistDiag)
    matDiag = mat.diagonal()
    hfEnergy = matDiag.min()
    indices = numpy.nonzero(abs(matDiag - hfEnergy) < 1e-8)[0]
    normalization = 1./(len(indices)**(1/2))
    fullSpace = mat.shape[0]
    matDat = [normalization]*len(indices)
    hfState = scipy.sparse.csc_matrix((matDat,(indices,[0]*len(indices))),shape=(fullSpace,1),dtype=numpy.complex128)
    return hfState, indices

In [29]:
for speciesname in molecule_data:
    print(speciesname)
    atoms, coords, mult, charge, basis, sector = molecule_data[speciesname].values()
    
    construct_mol=mol.construct_molecule(atoms=atoms, 
                                   coords=coords, 
                                   charge=charge, 
                                   multiplicity=mult, 
                                   basis=basis, 
                                   taper=False)
    
    ham_untapered = construct_mol['hamiltonian']
    untap_hf_config = construct_mol['hf_config']
    try:
        construct_mol=mol.construct_molecule(atoms=atoms, 
                                       coords=coords, 
                                       charge=charge, 
                                       multiplicity=mult, 
                                       basis=basis, 
                                       taper=True, 
                                       sym_sector=sector)
        ham_tapered_qiskit = construct_mol['hamiltonian']

        ham_untapered_indexed = qonvert.dict_to_list_index(ham_untapered)
        #taper.hfDegenerateGroundState(ham_untapered_indexed)
        hf_index_andrew = hfDegenerateGroundState(ham_untapered_indexed)[1]
        hf_index_qiskit = bit.bin_to_int(untap_hf_config)
        print('HF index:', hf_index_andrew, hf_index_qiskit)
        
        ham_tapered_andrew_1 = qonvert.index_list_to_dict(taper.taperOplist(oplist=ham_untapered_indexed, hfStateIndex=hf_index_andrew[0]))
        ham_tapered_andrew_2 = qonvert.index_list_to_dict(taper.taperOplist(oplist=ham_untapered_indexed, hfStateIndex=hf_index_qiskit))

        ham_untap_mat = qonvert.dict_to_WeightedPauliOperator(ham_untapered).to_matrix()
        ham_tap_1_mat = qonvert.dict_to_WeightedPauliOperator(ham_tapered_qiskit).to_matrix()
        ham_tap_2_1_mat = qonvert.dict_to_WeightedPauliOperator(ham_tapered_andrew_1).to_matrix()
        ham_tap_2_2_mat = qonvert.dict_to_WeightedPauliOperator(ham_tapered_andrew_2).to_matrix()


        print('Ground state energy untapered:          ',la.get_ground_state(ham_untap_mat)[0])
        print('Ground state energy tapered with Qiskit:',la.get_ground_state(ham_tap_1_mat)[0])
        print('Ground state energy tapered with Andrew (andrew hf):',la.get_ground_state(ham_tap_2_1_mat)[0])
        print('Ground state energy tapered with Andrew (qiskit hf):',la.get_ground_state(ham_tap_2_2_mat)[0])

        print('Qiskit num qubits: ',len(list(ham_tapered_qiskit.keys())[0]))
        print('Andrew num qubits: ',len(list(ham_tapered_andrew_1.keys())[0]), '\n')
    except:
        print('<!> Tapering went wrong \n')

H2_3-21G_SINGLET
Attempting to taper 8 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (-1, -1, 1), tapering successful!

HF index: [192] 136
Ground state energy untapered:           -1.1478774014540079
Ground state energy tapered with Qiskit: -1.147877401454005
Ground state energy tapered with Andrew (andrew hf): -0.23679831458316108
Ground state energy tapered with Andrew (qiskit hf): -0.23679831458316297
Qiskit num qubits:  5
Andrew num qubits:  5 

H6_STO-3G_SINGLET
Attempting to taper 12 --> 9 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (-1, -1, 1), tapering successful!

HF index: [4032] 3640
Ground state energy untapered:           -3.2360662798923574
Ground state energy tapered with Qiskit: -3.236066279892362
Ground state energy tapered with Andrew (andrew hf): -2.8848852002028815
Ground state energy tapered with Andrew (qiskit hf): -2.884885200202895
Qiskit num qubits:  9
Andrew num qubits:  9 

H2_6-31G_SINGLET
Attempting to 

Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (-1, -1, -1, -1, -1), tapering successful!

HF index: [ 981 1002] 924
Ground state energy untapered:           -53.71901016259449
Ground state energy tapered with Qiskit: -53.719010162593925
Ground state energy tapered with Andrew (andrew hf): -53.18116795713685
Ground state energy tapered with Andrew (qiskit hf): -53.1811679571369
Qiskit num qubits:  5
Andrew num qubits:  5 

OH+_STO-3G_SINGLET
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energy mismatch with target problem in sector (1, 1, 1, 1), trying another...
Energy mismatch with target problem in sector (1, 1, 1, -1), trying another...
Energy mismatch with target problem in sector (1, 1, -1, 1), trying another...
Energies match in sector (-1, 1, 1, 1), tapering successful!

HF index: [4060 4076 4084 4088] 3900
Ground state energy untapered:           -74.39362962132623
Ground state energy tapered with Qi

# Qiskit tapering implementation

In [8]:
construct_mol=mol.construct_molecule(atoms=atoms, 
                                   coords=coords, 
                                   charge=charge, 
                                   multiplicity=mult, 
                                   basis=basis, 
                                   taper=False)
ham_untapered = construct_mol['hamiltonian']
untap_hf_config = construct_mol['hf_config']

In [57]:
construct_mol=mol.construct_molecule(atoms=atoms, 
                                   coords=coords, 
                                   charge=charge, 
                                   multiplicity=mult, 
                                   basis=basis, 
                                   taper=True, 
                                   sym_sector=sector)
ham_tapered_qiskit = construct_mol['hamiltonian']

Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, -1, -1), tapering successful!



In [68]:
ham_untap_mat = qonvert.dict_to_WeightedPauliOperator(ham_untapered).to_matrix()
ham_tap_1_mat   = qonvert.dict_to_WeightedPauliOperator(ham_tapered_qiskit).to_matrix()

# Original tapering implementation

In [69]:
ham_untapered_indexed = qonvert.dict_to_list_index(ham_untapered)

In [70]:
taper.hfDegenerateGroundState(ham_untapered_indexed)

[4092]


<4096x1 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Column format>

In [71]:
hf_index = 4092#bit.bin_to_int(untap_hf_config)
ham_tapered_andrew = qonvert.index_list_to_dict(taper.taperOplist(oplist=ham_untapered_indexed, hfStateIndex=hf_index))
ham_tap_2_mat = qonvert.dict_to_WeightedPauliOperator(ham_tapered_andrew).to_matrix()

# Compare

In [72]:
print('Ground state energy untapered:          ',la.get_ground_state(ham_untap_mat)[0])
print('Ground state energy tapered with Qiskit:',la.get_ground_state(ham_tap_1_mat)[0])
print('Ground state energy tapered with Andrew:',la.get_ground_state(ham_tap_2_mat)[0])

Ground state energy untapered:           -98.60330177725983
Ground state energy tapered with Qiskit: -98.60330177725861
Ground state energy tapered with Andrew: -98.13175135661567


In [73]:
len(list(ham_tapered_andrew.keys())[0]), len(list(ham_tapered_qiskit.keys())[0])

(8, 8)